In [ ]:
%cd /app

import os
import pandas as pd
import datetime
import numpy as np
from pandas import IndexSlice as idx

In [ ]:
df_rbob = pd.read_excel("data/rbob_price.xls", sheet_name = "Data 1", index_col="date")
df_ho = pd.read_excel("data/heating_oil_price.xls", sheet_name="Data 1", index_col="date")
df_wti = pd.read_excel("data/wti_cushing_price.xls", sheet_name="Data 1", index_col="date")

price_df = pd.concat([df_rbob, df_wti, df_ho], axis=1, join="inner")

In [ ]:
price_df['321_spread'] = (2/3)*price_df['rbob_m1']*42 + (1/3)*price_df['ho_m1']*42 - price_df['wti_cush_spot']

In [ ]:
price_df.columns

In [ ]:
def compute_sd_distance(s: pd.Series, window=90):
    price_distance = s - s.rolling(window).mean()
    sd_distance = price_distance / s.rolling(3*window).std()
    return sd_distance

In [ ]:
for x in ['321_spread', 'rbob_m1', 'wti_cush_spot', 'ho_m1']:
    price_df[x + '_sd_distance'] = compute_sd_distance(price_df[x])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))

plt.plot(price_df.index, price_df['rbob_m1_sd_distance'], label='RBOB M1 SD Distance')
plt.plot(price_df.index, price_df['wti_cush_spot_sd_distance'], label='WTI Cush Spot SD Distance')
plt.plot(price_df.index, price_df['ho_m1_sd_distance'], label='HO M1 SD Distance')

plt.xlabel('Date')
plt.ylabel('SD Distance')
plt.title('SD Distances for RBOB M1, WTI Cush Spot, and HO M1')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
price_df.to_csv("data/price_data.csv")